In [1]:
%cd mmdetection

# !pip install -q --upgrade wandb
# !pip3 install openmim
# !mim install mmcv-full
# !pip install -e .

/aiffel/aiffel/aimmo/mmdetection


In [2]:
from mmdet.apis import init_detector, inference_detector, show_result_pyplot
import mmcv
from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets.builder import DATASETS
from mmdet.datasets.coco import CocoDataset,CustomDataset
from mmdet.apis import set_random_seed
import os.path as osp
import os
import numpy as np

import wandb
print(wandb.__version__)

/opt/conda/lib/python3.9/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


0.13.6


In [3]:
# # 완디비 로그인
wandb.login(key= '16988853f89d035c131fd35c4ad72529e9a1c09d')

wandb: Currently logged in as: choihoonsung33 (tesk-b). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /aiffel/.netrc


True

In [4]:
wandb.init(project="test-project", entity="tesk-b")

In [5]:
@DATASETS.register_module(force=True)
class Aimmo(CustomDataset):
    CLASSES = ('car', 'truck', 'bus','pedestrian')
    def load_annotations(self, ann_file):
        cat2label = {k:i for i, k in enumerate(self.CLASSES)}
        image_list = mmcv.list_from_file(self.ann_file)
        data_infos = []
    
        for image_id in image_list:
            filename = os.path.splitext(image_id)[0]#os.path.splitext('{0:}/{1:}'.format(img_prefix, image_id))[0]
            #image = cv2.imread(filename)
            data_info = {'filename': filename,
                         'width': 1920, 
                         'height': 1024}
            label_prefix = self.img_prefix.replace('image', 'middle_txt')
            lines = mmcv.list_from_file(osp.join(label_prefix, str(image_id)))

            content = [line.strip().split(' ') for line in lines]
            bbox_names = [x[0] for x in content]
            bboxes = [ [float(info) for info in x[1:5]] for x in content]

            gt_bboxes = []
            gt_labels = []
            gt_bboxes_ignore = []
            gt_labels_ignore = []

            for bbox_name, bbox in zip(bbox_names, bboxes):
                if bbox_name in cat2label:
                    gt_bboxes.append(bbox)
                    gt_labels.append(cat2label[bbox_name])
                else:
                    gt_bboxes_ignore.append(bbox)
                    gt_labels_ignore.append(-1)
            
            data_anno = {
              'bboxes': np.array(gt_bboxes, dtype=np.float32).reshape(-1, 4),
              'labels': np.array(gt_labels, dtype=np.compat.long),
              'bboxes_ignore': np.array(gt_bboxes_ignore, dtype=np.float32).reshape(-1, 4),
              'labels_ignore': np.array(gt_labels_ignore, dtype=np.compat.long)
            }
            data_info.update(ann=data_anno)
            data_infos.append(data_info)
        return data_infos

In [6]:
# # # We download the pre-trained checkpoints for inference and finetuning.
# # !mkdir checkpoints
# !wget -c https://download.openmmlab.com/mmdetection/v2.0/yolo/yolov3_d53_fp16_mstrain-608_273e_coco/yolov3_d53_fp16_mstrain-608_273e_coco_20210517_213542-4bc34944.pth \
#       -O checkpoints/yolov3_d53_fp16_mstrain-608_273e_coco.pth

In [7]:
config = 'configs/yolo/yolov3_d53_mstrain-608_273e_coco.py'
checkpoint_file = 'yolov3_d53_fp16_mstrain-608_273e_coco.pth'
cfg = Config.fromfile(config)

cfg.load_from = 'checkpoints/yolov3_d53_fp16_mstrain-608_273e_coco.pth'
cfg.dataset_type = 'Aimmo'
cfg.data_root = 'data/'


cfg.data.train.type = 'Aimmo'
cfg.data.train.data_root = 'data/'
cfg.data.train.ann_file = 'middle_train.txt'
cfg.data.train.img_prefix = 'image'

cfg.data.val.type = 'Aimmo'
cfg.data.val.data_root = 'data/'
cfg.data.val.ann_file = 'middle_val.txt'
cfg.data.val.img_prefix = 'image'

cfg.data.test.type = 'Aimmo'
cfg.data.test.data_root = 'data/'
cfg.data.test.ann_file = 'middle_val.txt'
cfg.data.test.img_prefix = 'image'

cfg.model.bbox_head.num_classes = 4

cfg.train_pipeline[4]['img_scale']=(960, 960)
cfg.test_pipeline[1]['img_scale']=(960, 960)
cfg.data.train.pipeline[4]['img_scale']=(960, 960)
cfg.data.val.pipeline[1]['img_scale']=(960, 960)
cfg.data.test.pipeline[1]['img_scale']=(960,960)

cfg.work_dir = 'yolo3'

cfg.data.samples_per_gpu = 2
cfg.data.workers_per_gpu = 0
cfg['device'] = 'cuda'

cfg.runner['max_epochs'] = 15
cfg.lr_config.policy='step'
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)
cfg.device='cuda'
cfg.evaluation.metric='mAP'

cfg.log_config.interval = 100
cfg.evaluation.interval = 1
cfg.checkpoint_config.interval = 1

print(cfg.pretty_text)

checkpoint_config = dict(interval=1)
log_config = dict(interval=100, hooks=[dict(type='TextLoggerHook')])
custom_hooks = [dict(type='NumClassCheckHook')]
dist_params = dict(backend='nccl')
log_level = 'INFO'
load_from = 'checkpoints/yolov3_d53_fp16_mstrain-608_273e_coco.pth'
resume_from = None
workflow = [('train', 1)]
opencv_num_threads = 0
mp_start_method = 'fork'
auto_scale_lr = dict(enable=False, base_batch_size=64)
model = dict(
    type='YOLOV3',
    backbone=dict(
        type='Darknet',
        depth=53,
        out_indices=(3, 4, 5),
        init_cfg=dict(type='Pretrained', checkpoint='open-mmlab://darknet53')),
    neck=dict(
        type='YOLOV3Neck',
        num_scales=3,
        in_channels=[1024, 512, 256],
        out_channels=[512, 256, 128]),
    bbox_head=dict(
        type='YOLOV3Head',
        num_classes=4,
        in_channels=[512, 256, 128],
        out_channels=[1024, 512, 256],
        anchor_generator=dict(
            type='YOLOAnchorGenerator',
            b

In [8]:
cfg.log_config.hooks = [
    dict(type='TextLoggerHook'),
    dict(type='WandbLoggerHook',
         init_kwargs={'project': 'test-project', 'entity' : 'tesk-b', 'name' : 'yolo3'},)] 

In [9]:
#build_dataset(cfg.data.train)

/aiffel/aiffel/aimmo/mmdetection/mmdet/datasets/custom.py:181: UserWarning: CustomDataset does not support filtering empty gt images.
  warnings.warn(



Aimmo Train dataset with number of images 20904, and instance counts: 
+----------+--------+-----------+-------+----------+-------+----------------+-------+---------------+-------+
| category | count  | category  | count | category | count | category       | count | category      | count |
+----------+--------+-----------+-------+----------+-------+----------------+-------+---------------+-------+
| 0 [car]  | 116578 | 1 [truck] | 27425 | 2 [bus]  | 16203 | 3 [pedestrian] | 28342 | -1 background | 10    |
+----------+--------+-----------+-------+----------+-------+----------------+-------+---------------+-------+

In [10]:
# Build dataset
datasets = [build_dataset(cfg.data.train)]

# Build the detector
model = build_detector(cfg.model)
# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_detector(model, datasets, cfg, distributed=False, validate=True)

2022-12-10 12:29:06,081 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.
2022-12-10 12:29:08,817 - mmdet - INFO - load checkpoint from local path: checkpoints/yolov3_d53_fp16_mstrain-608_273e_coco.pth
2022-12-10 12:29:09,926 - mmdet - WARNING - The model and loaded state dict do not match exactly

size mismatch for bbox_head.convs_pred.0.weight: copying a param with shape torch.Size([255, 1024, 1, 1]) from checkpoint, the shape in current model is torch.Size([27, 1024, 1, 1]).
size mismatch for bbox_head.convs_pred.0.bias: copying a param with shape torch.Size([255]) from checkpoint, the shape in current model is torch.Size([27]).
size mismatch for bbox_head.convs_pred.1.weight: copying a param with shape torch.Size([255, 512, 1, 1]) from checkpoint, the shape in current model is torch.Size([27, 512, 1, 1]).
size mismatch for bbox_head.convs_pred.1.bias: copying a param with shape torch.Size([255]) from checkpoint, the shape in current model is torch.Size([27

2022-12-10 12:30:40,483 - mmdet - INFO - Epoch [1][100/10452]	lr: 1.445e-04, eta: 1 day, 13:09:22, time: 0.854, data_time: 0.349, memory: 4602, loss_cls: 40.7286, loss_conf: 11973.4242, loss_xy: 43.7573, loss_wh: 21.9968, loss: 12079.9068, grad_norm: 24331.5849
2022-12-10 12:32:00,935 - mmdet - INFO - Epoch [1][200/10452]	lr: 1.895e-04, eta: 1 day, 12:03:43, time: 0.805, data_time: 0.330, memory: 4602, loss_cls: 32.7492, loss_conf: 79.1494, loss_xy: 46.0363, loss_wh: 5.0450, loss: 162.9798, grad_norm: 543.5933
2022-12-10 12:33:21,467 - mmdet - INFO - Epoch [1][300/10452]	lr: 2.345e-04, eta: 1 day, 11:41:39, time: 0.805, data_time: 0.327, memory: 4602, loss_cls: 25.8836, loss_conf: 54.6027, loss_xy: 46.4549, loss_wh: 4.1833, loss: 131.1246, grad_norm: 576.1119
2022-12-10 12:34:44,334 - mmdet - INFO - Epoch [1][400/10452]	lr: 2.795e-04, eta: 1 day, 11:45:09, time: 0.829, data_time: 0.324, memory: 4602, loss_cls: 20.2543, loss_conf: 46.9012, loss_xy: 39.9353, loss_wh: 3.7418, loss: 110.83

2022-12-10 13:16:30,703 - mmdet - INFO - Epoch [1][3400/10452]	lr: 1.000e-03, eta: 1 day, 11:31:57, time: 0.836, data_time: 0.339, memory: 4602, loss_cls: 9.4299, loss_conf: 37.4085, loss_xy: 35.2700, loss_wh: 2.1349, loss: 84.2432, grad_norm: 494.5412
2022-12-10 13:17:55,247 - mmdet - INFO - Epoch [1][3500/10452]	lr: 1.000e-03, eta: 1 day, 11:31:24, time: 0.845, data_time: 0.338, memory: 4602, loss_cls: 10.4630, loss_conf: 40.5147, loss_xy: 38.7517, loss_wh: 2.5402, loss: 92.2696, grad_norm: 610.3196
2022-12-10 13:19:17,586 - mmdet - INFO - Epoch [1][3600/10452]	lr: 1.000e-03, eta: 1 day, 11:29:14, time: 0.823, data_time: 0.336, memory: 4602, loss_cls: 9.4740, loss_conf: 41.2029, loss_xy: 39.4724, loss_wh: 2.3898, loss: 92.5391, grad_norm: 463.9383
2022-12-10 13:20:43,029 - mmdet - INFO - Epoch [1][3700/10452]	lr: 1.000e-03, eta: 1 day, 11:29:15, time: 0.854, data_time: 0.346, memory: 4602, loss_cls: 9.5956, loss_conf: 38.1075, loss_xy: 36.7623, loss_wh: 2.7724, loss: 87.2377, grad_no

2022-12-10 14:02:21,683 - mmdet - INFO - Epoch [1][6700/10452]	lr: 1.000e-03, eta: 1 day, 10:45:38, time: 0.818, data_time: 0.328, memory: 4602, loss_cls: 8.3480, loss_conf: 36.4644, loss_xy: 39.2211, loss_wh: 1.9769, loss: 86.0105, grad_norm: 365.5423
2022-12-10 14:03:45,037 - mmdet - INFO - Epoch [1][6800/10452]	lr: 1.000e-03, eta: 1 day, 10:44:14, time: 0.834, data_time: 0.333, memory: 4602, loss_cls: 7.0349, loss_conf: 36.4846, loss_xy: 37.6509, loss_wh: 1.8769, loss: 83.0473, grad_norm: 377.6648
2022-12-10 14:05:07,133 - mmdet - INFO - Epoch [1][6900/10452]	lr: 1.000e-03, eta: 1 day, 10:42:23, time: 0.821, data_time: 0.334, memory: 4602, loss_cls: 7.4239, loss_conf: 35.5972, loss_xy: 38.5036, loss_wh: 1.8379, loss: 83.3627, grad_norm: 389.4033
2022-12-10 14:06:28,798 - mmdet - INFO - Epoch [1][7000/10452]	lr: 1.000e-03, eta: 1 day, 10:40:23, time: 0.817, data_time: 0.329, memory: 4602, loss_cls: 8.4450, loss_conf: 39.8849, loss_xy: 42.2648, loss_wh: 2.2092, loss: 92.8039, grad_nor

2022-12-10 14:48:09,017 - mmdet - INFO - Epoch [1][10000/10452]	lr: 1.000e-03, eta: 1 day, 9:58:44, time: 0.842, data_time: 0.334, memory: 4602, loss_cls: 6.7073, loss_conf: 33.4790, loss_xy: 36.1652, loss_wh: 1.9967, loss: 78.3483, grad_norm: 336.2889
2022-12-10 14:49:29,012 - mmdet - INFO - Epoch [1][10100/10452]	lr: 1.000e-03, eta: 1 day, 9:56:32, time: 0.800, data_time: 0.332, memory: 4602, loss_cls: 9.4217, loss_conf: 40.2229, loss_xy: 42.2724, loss_wh: 2.3669, loss: 94.2839, grad_norm: 396.3341
2022-12-10 14:50:52,811 - mmdet - INFO - Epoch [1][10200/10452]	lr: 1.000e-03, eta: 1 day, 9:55:16, time: 0.838, data_time: 0.334, memory: 4602, loss_cls: 8.0573, loss_conf: 39.3982, loss_xy: 38.6211, loss_wh: 2.0149, loss: 88.0915, grad_norm: 462.6934
2022-12-10 14:52:17,111 - mmdet - INFO - Epoch [1][10300/10452]	lr: 1.000e-03, eta: 1 day, 9:54:06, time: 0.843, data_time: 0.344, memory: 4602, loss_cls: 8.5855, loss_conf: 34.9051, loss_xy: 36.5542, loss_wh: 1.8199, loss: 81.8647, grad_nor

...

[>>>>>>>>>>>>>>>>>>>>>>>>>>>] 2323/2323, 4.8 task/s, elapsed: 482s, ETA:     0s
---------------iou_thr: 0.5---------------


2022-12-11 21:28:59,776 - mmdet - INFO - 
+------------+-------+-------+--------+-------+
| class      | gts   | dets  | recall | ap    |
+------------+-------+-------+--------+-------+
| car        | 13124 | 36544 | 0.965  | 0.931 |
| truck      | 3084  | 19883 | 0.926  | 0.813 |
| bus        | 1786  | 10810 | 0.943  | 0.856 |
| pedestrian | 3063  | 21781 | 0.854  | 0.657 |
+------------+-------+-------+--------+-------+
| mAP        |       |       |        | 0.814 |
+------------+-------+-------+--------+-------+
2022-12-11 21:28:59,834 - mmdet - INFO - Epoch(val) [13][2323]	AP50: 0.8140, mAP: 0.8141
2022-12-11 21:30:24,677 - mmdet - INFO - Epoch [14][100/10452]	lr: 1.000e-03, eta: 4:45:49, time: 0.848, data_time: 0.356, memory: 4614, loss_cls: 4.6919, loss_conf: 27.3872, loss_xy: 37.8594, loss_wh: 1.2568, loss: 71.1953, grad_norm: 226.4397
2022-12-11 21:31:48,179 - mmdet - INFO - Epoch [14][200/10452]	lr: 1.000e-03, eta: 4:44:26, time: 0.835, data_time: 0.339, memory: 4614, loss_cl

2022-12-11 22:13:30,650 - mmdet - INFO - Epoch [14][3200/10452]	lr: 1.000e-03, eta: 4:03:17, time: 0.845, data_time: 0.339, memory: 4614, loss_cls: 4.9404, loss_conf: 26.5885, loss_xy: 36.5094, loss_wh: 1.0498, loss: 69.0883, grad_norm: 221.3311
2022-12-11 22:14:52,648 - mmdet - INFO - Epoch [14][3300/10452]	lr: 1.000e-03, eta: 4:01:55, time: 0.820, data_time: 0.332, memory: 4614, loss_cls: 4.5652, loss_conf: 29.1727, loss_xy: 39.2512, loss_wh: 1.1719, loss: 74.1610, grad_norm: 237.6598
2022-12-11 22:16:16,189 - mmdet - INFO - Epoch [14][3400/10452]	lr: 1.000e-03, eta: 4:00:32, time: 0.835, data_time: 0.341, memory: 4614, loss_cls: 3.4876, loss_conf: 25.2847, loss_xy: 34.2089, loss_wh: 0.8867, loss: 63.8678, grad_norm: 223.9425
2022-12-11 22:17:39,608 - mmdet - INFO - Epoch [14][3500/10452]	lr: 1.000e-03, eta: 3:59:10, time: 0.834, data_time: 0.335, memory: 4614, loss_cls: 4.2792, loss_conf: 29.0933, loss_xy: 38.8788, loss_wh: 1.1061, loss: 73.3573, grad_norm: 231.9837
2022-12-11 22:19

2022-12-11 23:00:33,590 - mmdet - INFO - Epoch [14][6600/10452]	lr: 1.000e-03, eta: 3:16:36, time: 0.851, data_time: 0.342, memory: 4614, loss_cls: 4.5835, loss_conf: 25.1832, loss_xy: 35.1232, loss_wh: 0.9644, loss: 65.8543, grad_norm: 224.6470
2022-12-11 23:01:55,507 - mmdet - INFO - Epoch [14][6700/10452]	lr: 1.000e-03, eta: 3:15:13, time: 0.819, data_time: 0.323, memory: 4614, loss_cls: 4.4999, loss_conf: 28.2998, loss_xy: 38.3411, loss_wh: 1.3019, loss: 72.4427, grad_norm: 224.6463
2022-12-11 23:03:16,719 - mmdet - INFO - Epoch [14][6800/10452]	lr: 1.000e-03, eta: 3:13:51, time: 0.812, data_time: 0.325, memory: 4614, loss_cls: 3.7406, loss_conf: 27.6043, loss_xy: 37.0833, loss_wh: 1.1478, loss: 69.5759, grad_norm: 215.8550
2022-12-11 23:04:38,152 - mmdet - INFO - Epoch [14][6900/10452]	lr: 1.000e-03, eta: 3:12:28, time: 0.814, data_time: 0.331, memory: 4614, loss_cls: 3.9985, loss_conf: 26.8788, loss_xy: 37.0058, loss_wh: 1.0965, loss: 68.9796, grad_norm: 205.2265
2022-12-11 23:05

KeyboardInterrupt: 